In [ ]:
# But first, let's install the dependecies.
!pip install -r requirements.txt

In [1]:
import os
from dotenv import load_dotenv
from video_prompter import get_connection

# TODO: setup .env file
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

# connect to VideoDB
conn = get_connection()

### ----- Upload a fresh video --------- #####
def fresh_video(url):
    video = conn.upload(url=url)
    #index spoken content in the video
    video.index_spoken_words()
    return video

#### ------ run the prompt on video --------####
def videodb_prompter(video_id, prompt):
    video = get_video(video_id)
    #get all the segment of videos that are
    return video_prompter(video, prompt)

In [2]:
from videodb import play_stream

# ----Existing video case ----

# TODO: replace with your video id
# # video_id = "m-replace-with-your-video-id-24-7"
# video_id = 'm-beb3ee4a-3169-4cb6-8dc1-beee73eb00a1'
# video_id = 'm-08caaef7-1680-4ca7-9090-38d11668bca9'
video_id = "m-3b9d3378-d79a-4765-9144-fa96da9645e8"


# ---- Fresh video case ----
# url = "https://youtu.be/pAMy7IhOVQE?si=iNvbJE02Pf8XXGd8"
# video = fresh_video(url)
# video_id = video.id


#watch the original video
video = conn.get_collection().get_video(video_id)
video.player_url

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/bd38e2b1-fd23-4629-935e-5d38a80738fe.m3u8'

In [ ]:
# Create new scene index
from videodb import IndexType
from videodb import SceneExtractionType
 
index_id = video.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={"time":10, "select_frames": ['first']},
    prompt="all the actions taken by charchter in the frame",
    # callback_url=callback_url,
)

# Wait to Indexing to finish
# scene_index = video.get_scene_index()
# print(scene_index)

# #Use existing scene index

In [3]:
index_id = video.list_scene_index()
print(index_id)

[{'name': 'My Custom Annotations#1', 'scene_index_id': '0229a1d0959a0c85', 'status': 'done'}, {'name': 'Scene Index 2024-07-19 10:39', 'scene_index_id': '49b2dfa77e3c55e9', 'status': 'done'}, {'name': 'My Custom Annotations#1', 'scene_index_id': 'd1ae348b01211969', 'status': 'done'}, {'name': 'Scene Index 2024-07-18 06:43', 'scene_index_id': 'd49f6fbcc95a87d5', 'status': 'done'}]


In [4]:
scene_index = video.get_scene_index("0229a1d0959a0c85")
# print(scene_index)

In [5]:
transcript = video.get_transcript()

In [6]:
from video_prompter import multimodal_prompter


prompt = "a conversastion on religion"
matches = multimodal_prompter(transcript,scene_index, prompt)


Sendiing call to OPENAI 
       You are given visual and spoken information of the video of each second, and a transcipt of what's being spoken along with timestamp.
        Your task is to evaluate the data for relevance to the specified user prompt.
        Corelate visual and spoken content to find the relevant video segment.
        provide the start and end timestamps by analyse the full chunk and give longest matching timestamps. You can merge the 1 second chunks and transcripts to make continuous response.

        Multimodal Data:
        video: [{'visual': 'Detective Martin is being interviewed by the police.', 'spoken': 'Revival ministry, old time religion. You can imagine what Mister Charisma thought of that.', 'start': 0.0, 'end': 10.0}, {'visual': 'A religious gathering. People are praying and singing.', 'spoken': "What do you think the average iq of this group is, huh? Can you see Texas up there on your high horse? What do you know about these people? Just observation and

{'id': 'chatcmpl-9o4mNfdZfJI6nc2d5oEnP9NYThM0e', 'object': 'chat.completion', 'created': 1721721799, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{"result": {"visual": "A religious gathering. People are praying and singing.", "spoken": "What do you think the average iq of this group is, huh? Can you see Texas up there on your high horse? What do you know about these people? Just observation and deduction. See, a propensity for obesity, poverty, a yin for fairy tales. Folks putting what few bucks they do have in the little wicker baskets being passed around. I think it\'s safe to say that nobody here is gonna be splitting the atom, Marty. Can you imagine if people didn\'t believe things? They\'d get up to exact same thing they do now, just out in the open. Bull shit. It could be a fucking freak show of murder and debauchery and you know it. If the only thing keeping a person decent is the expectation of divine reward, then, brother, that t

In [9]:
# search matching video segments and watch the stream
from videodb import SearchType
from videodb.timeline import Timeline, VideoAsset, AudioAsset
from video_prompter import extract_clip_sentences


timeline = Timeline(conn)
for clip_sentences in extract_clip_sentences(matches):
    search_res = video.search(clip_sentences, search_type=SearchType.keyword)
    matched_segments = search_res.get_shots()
    
    #no exact match found.
    if len(matched_segments) == 0:
        continue

    #video segment
    video_shot = matched_segments[0]

    #Create a new Video Asset and add it to a timeline.
    timeline.add_inline(VideoAsset(asset_id=video.id, start=video_shot.start, end=video_shot.end))

Processed Item 0


In [10]:
stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://stream.videodb.io/v3/published/manifests/17712372-e786-4c87-8b2a-c3b4ad9d553e.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/17712372-e786-4c87-8b2a-c3b4ad9d553e.m3u8'

In [11]:
def construct_timeline_and_stream( selected_frames):
    timeline = Timeline(conn)
    
    print("Selected frames:", selected_frames)  

    for frame in selected_frames:
        print("Adding frame:", frame)  
        video_asset = VideoAsset(
            asset_id=video_id,
            start=frame['start_time'],
            end=frame['end_time']
        )
        timeline.add_inline(video_asset)
    
    print("Timeline created:", timeline)  
    
    stream_url = timeline.generate_stream()
    print("Stream URL:", stream_url)
    return play_stream(stream_url)


In [13]:
from video_prompter import merge_timeframes,extract_timestamps_multimodal


timeframes = extract_timestamps_multimodal(matches)
merged_timeframes = merge_timeframes(timeframes)
construct_timeline_and_stream(merged_timeframes)

Extracted timeframe 0: start=10.0, end=100.0
Selected frames: [{'start_time': 10.0, 'end_time': 100.0, 'sentence': None}]
Adding frame: {'start_time': 10.0, 'end_time': 100.0, 'sentence': None}
Timeline created: <videodb.timeline.Timeline object at 0x108a85390>
Stream URL: https://stream.videodb.io/v3/published/manifests/e4cc62ee-fa4b-4d7d-a77f-1808fd33cbc7.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/e4cc62ee-fa4b-4d7d-a77f-1808fd33cbc7.m3u8'